In [4]:
import openai

from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyMuPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

from langchain.chat_models import ChatOpenAI

from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferMemory

# used to create the retrieval tool
from langchain.agents.agent_toolkits import create_retriever_tool

# used to create the memory
from langchain.memory import ConversationBufferMemory

# used to create the prompt template
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema import SystemMessage
from langchain.prompts import MessagesPlaceholder

# used to create the agent executor
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor

In [5]:
# Retrieve API key
#os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')
from dotenv import load_dotenv
import os
load_dotenv()  
openai.api_key = os.getenv("OPENAI_API_KEY")

In [163]:
# load PDF documents using PyMuPDF
loader = DirectoryLoader('content/', glob="**/*.pdf", show_progress=True, loader_cls=PyMuPDFLoader)
docs = loader.load()

100%|██████████| 2/2 [00:01<00:00,  1.15it/s]


In [108]:
# chunk pdf texts and create Chroma vector db
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=100)
documents = text_splitter.split_documents(docs)
db = Chroma.from_documents(documents, OpenAIEmbeddings())


In [116]:
retriever = db.as_retriever(search_type="mmr")

tool = create_retriever_tool(
    retriever,
    "search_deep_learning_textbooks",
    "Searches and returns documents regarding deep learning and NLP",
)
tools = [tool]

In [117]:
memory_key = "history"
memory = ConversationBufferMemory(memory_key=memory_key, return_messages=True)

In [154]:
learning_style = "Visual: Bullet Points"

system_message = SystemMessage(
        content=(
            "You are a tutor for LIGN 167, a deep learning and NLP course at UCSD, with immense knowledge and experience in the field."
            "Answer students' questions based on your knowledge and conversation history. Do not make up answers."
            "If you do not know the answer to a question, just say \'I don't know\'."
            "If asked a question not relevant to deep learning or NLP, just say \'I cannot answer your question\'."
            "Tailor your response to match a " + learning_style + " learning style."
        )
)

prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[MessagesPlaceholder(variable_name=memory_key)]
    )

In [155]:
llm = ChatOpenAI(temperature = 0, model = "gpt-4-1106-preview")
agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)

In [156]:
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)

In [157]:
result = agent_executor({"input": "hi, im bob"})



> Entering new AgentExecutor chain...
Hello Bob! How can I assist you with deep learning or NLP today?

> Finished chain.


In [158]:
result = agent_executor({"input": "what is my name?"})



> Entering new AgentExecutor chain...
Your name is Bob. If you have any questions related to deep learning or NLP, feel free to ask!

> Finished chain.


In [159]:
result = agent_executor({"input": "what is a MLP?"})



> Entering new AgentExecutor chain...
- **MLP**: Multi-Layer Perceptron
- **Type**: Artificial Neural Network (ANN)
- **Architecture**: 
  - Input layer
  - One or more hidden layers
  - Output layer
- **Connections**: Fully connected (each neuron to every neuron in the next layer)
- **Data Flow**: Feedforward (no cycles or loops in the network)
- **Activation Function**: Non-linear (e.g., ReLU, sigmoid, tanh)
- **Learning**: Supervised (uses backpropagation)
- **Purpose**: Classification, regression (non-sequential data)
- **Limitation**: Cannot process sequential data as RNNs do

> Finished chain.


In [162]:
result = agent_executor({"input": "how does it differ from an RNN?"})



> Entering new AgentExecutor chain...
- **MLP vs. RNN**:

1. **Architecture**:
   - MLP: Feedforward, no cycles
   - RNN: Recurrent, with cycles allowing information to persist

2. **Data Flow**:
   - MLP: One-way from input to output
   - RNN: Loops in the network enable sequence processing

3. **Temporal Dynamics**:
   - MLP: Treats each input independently
   - RNN: Maintains internal state for sequence dependency

4. **Use Cases**:
   - MLP: Static input-output mapping (e.g., image classification)
   - RNN: Sequential data (e.g., time series, language modeling)

5. **Memory**:
   - MLP: No memory of past inputs
   - RNN: Memory through recurrent connections

6. **Training**:
   - MLP: Generally simpler, backpropagation
   - RNN: More complex, backpropagation through time (BPTT)

7. **Challenges**:
   - MLP: May struggle with complex feature hierarchies
   - RNN: Vanishing and exploding gradient problems

8. **Variants**:
   - MLP: Fewer variants
   - RNN: Advanced variants like L

'- **MLP vs. RNN**:\n\n1. **Architecture**:\n   - MLP: Feedforward, no cycles\n   - RNN: Recurrent, with cycles allowing information to persist\n\n2. **Data Flow**:\n   - MLP: One-way from input to output\n   - RNN: Loops in the network enable sequence processing\n\n3. **Temporal Dynamics**:\n   - MLP: Treats each input independently\n   - RNN: Maintains internal state for sequence dependency\n\n4. **Use Cases**:\n   - MLP: Static input-output mapping (e.g., image classification)\n   - RNN: Sequential data (e.g., time series, language modeling)\n\n5. **Memory**:\n   - MLP: No memory of past inputs\n   - RNN: Memory through recurrent connections\n\n6. **Training**:\n   - MLP: Generally simpler, backpropagation\n   - RNN: More complex, backpropagation through time (BPTT)\n\n7. **Challenges**:\n   - MLP: May struggle with complex feature hierarchies\n   - RNN: Vanishing and exploding gradient problems\n\n8. **Variants**:\n   - MLP: Fewer variants\n   - RNN: Advanced variants like LSTM, GR

In [153]:
result = agent_executor({"input": "what date was UCSD founded?"})



> Entering new AgentExecutor chain...
I cannot answer your question.

> Finished chain.


In [93]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True, output_key='answer')

agent = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=db.as_retriever(search_type="mmr"), memory=memory, verbose=True
)

""" agent = initialize_agent(
    tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, memory=memory, verbose=True
) """

' agent = initialize_agent(\n    tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, memory=memory, verbose=True\n) '